In [2]:
import os
folder_path = 'Patna'  
files = os.listdir(folder_path)
num_files = len(files)
files.sort(reverse=True)
print(f"Number of file in folder: {num_files}")


Number of file in folder: 0


In [4]:
import pandas as pd
df=pd.read_csv("csvs/patna/patna_12_10.csv")
df

,meta_data,judgment,file_name
0,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,"Date : 29-08-2023\nHeard Mr. Salahuddin Khan, ...",Zeenat_Praveen_vs_The_Municipal_Building_Tribu...
1,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,"Date : 18-07-2023 Heard Mr. Mohammad Sufyan, l...",Zeenat_Imam_vs_The_State_Of_Bihar_on_18_July_2...
2,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE)\nDate : 12...,Zareen_Abdul_Hai_Kulsum_Abdul_Hai_vs_Union_Of_...
3,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE)\nDate : 11...,Zahid_Hussain_vs_The_State_Of_Bihar_on_11_Augu...
4,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 20-02-2023\nLearned counsel for the res...,Zafirul_Hasan_vs_The_State_Of_Bihar_on_20_Febr...
...,...,...,...
7542,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCrim...,(Per: HONOURABLE MR. JUSTICE V.N. SINHA)\nDate...,Dipak_Kumar_Desho_vs_The_State_Of_Bihar_on_23_...
7543,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 03-11-2015 Heard learned counsel for the...,Dinesh_Narain_Mallik_vs_The_Bihar_State_Power_...
7544,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 24-01-2017\nThe petitioners in this batc...,Dinesh_Kumar_Singh_Ors_vs_The_State_Of_Bihar_O...
7545,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCrim...,Date: 07-03-2017\nHeard learned counsel for th...,Dina_Nath_Chamar_Ors_vs_State_Of_Bihar_Anr_on_...


In [10]:
len(f)

7547

In [29]:
common=[]
for i in (files):
    if(i  in f):
        common.append(i)


In [53]:
import shutil
import os

# Define source and destination folders
source_folder = 'Patna'
destination_folder = '/sml2/Judgments/extracted/patna'

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)
files=os.listdir(source_folder)
# Move files
for filename  in extracted_files:
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)
    shutil.move(source_file, destination_file)

print("Files moved successfully!")


Files moved successfully!


In [54]:
import shutil
import os

# Define source and destination folders
source_folder = 'Patna'
destination_folder = '/sml2/Judgments/missing/patna'

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)
files=os.listdir(source_folder)
# Move files
for filename  in missing_files:
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)
    shutil.move(source_file, destination_file)

print("Files moved successfully!")


Files moved successfully!


In [30]:
print(len(common))

0


In [52]:
import pdfplumber
import pandas as pd
import os
import re

df = pd.DataFrame()

def extract_text_between_headings(pdf_path, start_heading, end_heading,formatted_name):
    try:
        # Open the PDF file
        pdf = pdfplumber.open(pdf_path)
    except FileNotFoundError:
        print(f"The file {pdf_path} was not found.")
        return "", ""
    except Exception as e:
        print(f"An error occurred while opening the file: {e}")
        return "", ""

    text = ""
    text1 = ""
    extracting = False
    extract2 = False
    flag = 0

    # Iterate through the pages
    for page in pdf.pages:
        page_text = page.extract_text()
        lines = page_text.split('\n')
    
        for line in lines:
            # Skip lines with only "=" characters
            if end_heading in line:
                extracting = True
            if extract2 and not re.match(r'Indian Kanoon', line) and len(line) > 1 and not re.match(formatted_name[:10], line) and not re.match(r'^=+$', line): 
                text1 += line + '\n'
            if extracting and len(line) > 1 and not re.match(r'Indian Kanoon', line) and not re.match(formatted_name[:10], line) and not re.match(r'^=+$', line):
                text += line + '\n'
            if start_heading == line or line == "CAV JUDGMENT" or line=="C.A.V. JUDGMENT" or line =="CAV J U D G M E N T" or line=="ORAL JUDGMENT":
                print(line)
                flag = 1
                extracting = False
                extract2 = True
            
    if flag == 0:
        return '', ''
            
    return text.strip(), text1.strip()


judgment = []
meta_data = []
missing_files = []
extracted_files = []
for i in files:
    pdf_path = 'Patna/' + i
    file_name_without_ext = os.path.splitext(i)[0]
    formatted_name = file_name_without_ext.replace('_', ' ')
    start_heading = 'ORAL JUDGMENT'
    end_heading = 'IN THE HIGH'
    extracted_text, extracted_judg = extract_text_between_headings(pdf_path, start_heading, end_heading,formatted_name)
    if extracted_text:
        print(i)
        extracted_files.append(i)
        judgment.append(extracted_judg)
        meta_data.append(extracted_text)
    else:
        missing_files.append(i)
        print(f"Could not extract text from file {i}")

df['meta_data'] = meta_data
df['judgment'] = judgment
df['file_name'] = extracted_files
df.to_csv('patna_extract1.csv', index=False)

ORAL JUDGMENT
Md_Yunus_vs_The_State_Of_Bihar_Ors_on_20_July_2018.PDF


CAV JUDGMENT
Md_Yunus_Md_Yunis_vs_The_State_Of_Bihar_on_11_October_2022.PDF
ORAL JUDGMENT
Md_Yaseen_Ansari_vs_The_State_Of_Bihar_Ors_on_6_September_2018.PDF
ORAL JUDGMENT
Md_Yaqub_Ansari_vs_The_Union_Of_India_Through_The_Geneal_on_5_July_2022.PDF
ORAL JUDGMENT
Md_Yahya_vs_The_State_Of_Bihar_Ors_on_6_July_2018.PDF
ORAL JUDGMENT
Md_Waziur_Rahaman_Lal_Babu_vs_State_Of_Bihar_Anr_on_6_March_2018.PDF
ORAL JUDGMENT
Md_Waziruddin_vs_The_State_Of_Bihar_Ors_on_24_August_2018.PDF
ORAL JUDGMENT
Md_Wazir_vs_The_State_Of_Bihar_Ors_on_10_September_2018.PDF
ORAL JUDGMENT
Md_Wasiur_Rahman_Anr_vs_The_State_Of_Bihar_Ors_on_25_April_2018.PDF
ORAL JUDGMENT
Md_Wasique_vs_State_Of_Bihar_on_9_March_2018.PDF
ORAL JUDGMENT
Md_Wasim_vs_The_Union_Of_India_on_28_July_2022.PDF
ORAL JUDGMENT
Md_Wasim_vs_The_State_Of_Bihar_Ors_on_29_October_2018.PDF
ORAL JUDGMENT
Md_Wasim_Uddin_vs_The_State_Of_Bihar_Ors_on_10_August_2018.PDF
Could not extract text from file Md_Wasim_Ors_vs_The_State_Of_Bihar_Ors_on_13_April_2018.PDF


In [47]:
df

,meta_data,judgment,file_name
0,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCRIM...,Date : 15-07-2020 Heard learned counsel for th...,Parparba_Md_Jawed_vs_The_State_Of_Bihar_on_15_...
1,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE)\nDate : 02...,Paropkar_Consultant_Private_Limited_vs_The_Sta...
2,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE) Date : 19-...,Paropkar_Consultant_Private_Limited_vs_The_Sta...
3,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 17-07-2018\nThis application has been fi...,Paroo_Prakhand_Samagra_Vikas_vs_Prakash_Singh_...
4,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCrim...,"Date: 10-01-2018\nAppellants Paro Singh, Arbin...",Paro_Singh_Ors_vs_The_State_Of_Bihar_on_10_Jan...
...,...,...,...
1951,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 18-05-2018\nThe petitioner by way of th...,Md_Zakaullah_Khan_vs_The_State_Of_Bihar_Ors_on...
1952,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCrim...,Date : 11-12-2018 Heard learned counsel for th...,Md_Zahid_Jahid_Alam_vs_The_State_Of_Bihar_on_1...
1953,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCIVI...,Date : 03-06-2020\nThis civil revision applica...,Md_Zahid_Hussain_vs_Sri_Babul_Kumar_Saha_on_3_...
1954,IN THE HIGH COURT OF JUDICATURE AT PATNA\nLett...,(Per: HONOURABLE MR. JUSTICE P. B. BAJANTHRI)\...,Md_Zafar_Sadique_And_Anr_vs_The_State_Of_Bihar...


In [94]:
s=extracted_files
len(s)

3054

In [49]:
import pandas as pd
import glob


all_files=['patna_extract1.csv','patna_extract2.csv','patna_extract3.csv']

# Read and concatenate all CSV files
df_list = (pd.read_csv(file) for file in all_files)
concatenated_df = pd.concat(df_list, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv("merged.csv", index=False)


In [50]:
concatenated_df

,meta_data,judgment,file_name
0,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,"Date : 20-06-2018\nHeard Mr. T.N.Maitin, learn...",_Dr_Shashi_Nath_Jha_vs_Kameshwar_Singh_Darbhan...
1,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 13-08-2018 Heard learned counsel for th...,_Dr_Saroj_Kumar_Jha_vs_The_State_Of_Bihar_Ors_...
2,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 26-10-2018\nHeard learned counsel for t...,_Dr_Saifullah_Khalid_vs_The_Chancellor_Of_Univ...
3,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 02-04-2018 Heard learned counsel for the...,_Dr_Bindeshwar_Paswan_vs_B_N_Mandal_University...
4,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE)\nDate : 05...,Zydus_Healthcare_Limited_vs_The_State_Of_Bihar...
...,...,...,...
11696,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 18-05-2018\nThe petitioner by way of th...,Md_Zakaullah_Khan_vs_The_State_Of_Bihar_Ors_on...
11697,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCrim...,Date : 11-12-2018 Heard learned counsel for th...,Md_Zahid_Jahid_Alam_vs_The_State_Of_Bihar_on_1...
11698,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCIVI...,Date : 03-06-2020\nThis civil revision applica...,Md_Zahid_Hussain_vs_Sri_Babul_Kumar_Saha_on_3_...
11699,IN THE HIGH COURT OF JUDICATURE AT PATNA\nLett...,(Per: HONOURABLE MR. JUSTICE P. B. BAJANTHRI)\...,Md_Zafar_Sadique_And_Anr_vs_The_State_Of_Bihar...


In [4]:
import pandas as pd
df=pd.read_csv("patna_15_9.csv")
df

,meta_data,judgment,file_name
0,IN THE HIGH COURT OF JUDICATURE AT PATNA\nMisc...,"Date: 20-07-2018\nOpposite party no. 7, Distri...",Md_Yunus_vs_The_State_Of_Bihar_Ors_on_20_July_...
1,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCRIM...,(Per: HONOURABLE MR. JUSTICE CHAKRADHARI SHARA...,Md_Yunus_Md_Yunis_vs_The_State_Of_Bihar_on_11_...
2,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 06-09-2018 Heard learned counsel for the...,Md_Yaseen_Ansari_vs_The_State_Of_Bihar_Ors_on_...
3,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Md. Yaqub Ansari vs The Union Of India Through...,Md_Yaqub_Ansari_vs_The_Union_Of_India_Through_...
4,IN THE HIGH COURT OF JUDICATURE AT PATNA\nLett...,(Per: HONOURABLE THE CHIEF JUSTICE) Date : 06-...,Md_Yahya_vs_The_State_Of_Bihar_Ors_on_6_July_2...
...,...,...,...
10802,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date: 21-08-2018 The present writ petition has...,A_P_S_Battery_Plate_Udyog_vs_The_State_Of_Biha...
10803,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE MR. JUSTICE RAJEEV RANJAN PRA...,A_K_Cable_Network_vs_The_State_Of_Bihar_Ors_on...
10804,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,(Per: HONOURABLE THE CHIEF JUSTICE) Date : 17-...,A_B_Enterprises_Partner_Sunil_Singh_vs_The_Sta...
10805,IN THE HIGH COURT OF JUDICATURE AT PATNA\nCivi...,Date : 06-01-2022 The matter is heard via vide...,864570011_Asi__Exe_Gyandeo_Singh_vs_The_Union_...
